In [1]:
import sqlite3
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
# import pmdarima as pm
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
# Connect to database
conn = sqlite3.connect('rossmann.db')
cursor = conn.cursor()

# Print table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('rossmann',)]


In [3]:
# Assign 'rossmann' table to Pandas DataFrame
sql = "SELECT * FROM rossmann"
df = pd.read_sql(sql, conn)

In [4]:
# source: chatgpt.com, prompted with 'use OneHotEncoder with min_freq'

def encode_with_minfreq(df, cols_to_encode, min_freq=0.05):
    """
    One-hot encode categorical columns using sklearn, 
    dropping rare categories below a frequency threshold.

    Parameters
    ----------
    df : pd.DataFrame
        Input data.
    cols_to_encode : list
        Categorical column names to encode.
    min_freq : float or int
        Minimum frequency for a category to get its own column.
        If float, interpreted as proportion of samples.
    """
    enc = OneHotEncoder(
        sparse_output=False,
        handle_unknown="ignore",
        min_frequency=min_freq
    )
    encoded = enc.fit_transform(df[cols_to_encode])
    encoded_cols = enc.get_feature_names_out(cols_to_encode)
    encoded_df = pd.DataFrame(encoded, columns=encoded_cols, index=df.index)
    res = pd.concat([df.drop(cols_to_encode, axis=1), encoded_df], axis=1)
    return res


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   index                      1017209 non-null  int64  
 1   Store                      1017209 non-null  int64  
 2   DayOfWeek                  1017209 non-null  int64  
 3   Date                       1017209 non-null  object 
 4   Sales                      1017209 non-null  int64  
 5   Customers                  1017209 non-null  int64  
 6   Open                       1017209 non-null  int64  
 7   Promo                      1017209 non-null  int64  
 8   StateHoliday               1017209 non-null  object 
 9   SchoolHoliday              1017209 non-null  int64  
 10  StoreType                  1017209 non-null  object 
 11  Assortment                 1017209 non-null  object 
 12  CompetitionDistance        1014567 non-null  float64
 13  CompetitionO

In [6]:
# Used for easier time series analysis
indexed_df = df.copy()
indexed_df['Date'] = pd.to_datetime(df['Date'])
indexed_df = indexed_df.set_index('Date')
indexed_df = indexed_df.sort_index()

In [7]:
indexed_df['CompetitionDistance'] = indexed_df['CompetitionDistance'].fillna(indexed_df['CompetitionDistance'].median())

In [8]:
indexed_df[['CompetitionOpenSinceYear', 'Promo2SinceYear']] = indexed_df[['CompetitionOpenSinceYear', 'Promo2SinceYear']].fillna(0, axis=1)
indexed_df['PromoInterval'] = indexed_df['PromoInterval'].fillna('Unknown')


In [9]:
indexed_df = indexed_df.drop(columns=['CompetitionOpenSinceYear', 'Promo2SinceYear', 'index', 'CompetitionDistance', 'SchoolHoliday'])

In [10]:
indexed_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,StoreType,Assortment,CompetitionOpenSinceMonth,Promo2,Promo2SinceWeek,PromoInterval
Date,,,,,,,,,,,,,
2013-01-01,1115,2,0,0,0,0,a,d,c,NaN,1,22.0,"Mar,Jun,Sept,Dec"
2013-01-01,379,2,0,0,0,0,a,d,a,NaN,0,NaN,Unknown
2013-01-01,378,2,0,0,0,0,a,a,c,8.0,0,NaN,Unknown
2013-01-01,377,2,0,0,0,0,a,a,c,6.0,1,18.0,"Feb,May,Aug,Nov"
2013-01-01,376,2,0,0,0,0,a,a,a,8.0,0,NaN,Unknown


In [11]:
to_encode = ['DayOfWeek', 'StateHoliday', 'StoreType', 'Assortment', 'PromoInterval', 'Open', 'Promo', 'Promo2']

In [12]:
del(df)
df = encode_with_minfreq(indexed_df, to_encode, min_freq=0.05)
del(indexed_df)

In [13]:
df.head()

,Store,Sales,Customers,CompetitionOpenSinceMonth,Promo2SinceWeek,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,...,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",PromoInterval_Unknown,Open_0,Open_1,Promo_0,Promo_1,Promo2_0,Promo2_1
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1115,0,0,NaN,22.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2013-01-01,379,0,0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2013-01-01,378,0,0,8.0,NaN,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
2013-01-01,377,0,0,6.0,18.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2013-01-01,376,0,0,8.0,NaN,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [14]:
import numpy as np
import pandas as pd
from typing import Dict, Optional, Tuple
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error


DEFAULT_ORDER = (1, 1, 1)
DEFAULT_SEASONAL = (1, 1, 1, 7)
DEFAULT_STEPS = 28
DEFAULT_MIN_LEN = 60


# ---------- helpers ----------
def _safe_rmse(y, yhat):
    return float(np.sqrt(mean_squared_error(y, yhat)))


def _mape(y, yhat):
    y = np.asarray(y, dtype=float)
    yhat = np.asarray(yhat, dtype=float)
    mask = y != 0

    return float(np.mean(np.abs((yhat[mask] - y[mask]) / y[mask])))

def _tae(y, yhat):
    y = np.asarray(y, dtype=float)
    yhat = np.asarray(yhat, dtype=float)
    mask = y != 0

    return float(np.abs((yhat[mask].sum() - y[mask].sum())))

# ---------- core ----------
def fit_store_onehot(
    df_store: pd.DataFrame,
    steps: int = DEFAULT_STEPS,
    order: Tuple[int, int, int] = DEFAULT_ORDER,
    seasonal: Tuple[int, int, int, int] = DEFAULT_SEASONAL,
    min_len: int = DEFAULT_MIN_LEN,
    agg_sales: str = "sum",
) -> Optional[Dict[str, object]]:
    """Fit SARIMA for a single store with pre-encoded one-hot columns."""
    work = (
        df_store.drop(columns=["index"], errors="ignore")
                .sort_index()
                .asfreq("D")
    )

    if "Sales" not in work or len(work) <= max(steps + 1, min_len):
        return None

    y = work["Sales"].astype(float)
    X = work.drop(columns=["Sales", "Store", "CompOpenDate", "Promo2OpenDate", "Customers"], errors="ignore")
    X = X.apply(pd.to_numeric, errors="coerce").fillna(0.0)

    y_tr, y_te = y.iloc[:-steps], y.iloc[-steps:]
    X_tr, X_te = X.iloc[:-steps], X.iloc[-steps:]

    mod = SARIMAX(
        y_tr,
        order=order,
        seasonal_order=seasonal,
        exog=X_tr,
        enforce_stationarity=True,
        enforce_invertibility=True,
    )
    res = mod.fit(disp=False)

    fc = res.get_forecast(steps=steps, exog=X_te).predicted_mean
    y_a, f_a = y_te.align(fc, join="inner")
    mask = ~(y_a.isna() | f_a.isna())
    y_a = y_a[mask].to_numpy().astype(float)
    f_a = f_a[mask].to_numpy().astype(float)

    # mae = mean_absolute_error(y_a, f_a)
    rmse = _safe_rmse(y_a, f_a)
    mape = _mape(y_a, f_a)
    tae = _tae(y_a, f_a)

    return {
        "result": res,
        "forecast": fc,
        "metrics": {
            # "MAE": mae,
            "TAE": tae,
            "RMSE": rmse,
            "MAPE": mape,
            "AIC": res.aic,
            "BIC": res.bic,
        },
        "y_test": y_te,
    }


def fit_all_stores_onehot(
    df: pd.DataFrame,
    steps: int = DEFAULT_STEPS,
    order: Tuple[int, int, int] = DEFAULT_ORDER,
    seasonal: Tuple[int, int, int, int] = DEFAULT_SEASONAL,
    min_len: int = DEFAULT_MIN_LEN,
    agg_sales: str = "sum",
) -> Tuple[Dict[int, Dict[str, object]], pd.DataFrame]:
    """Run SARIMA per store on pre-encoded data."""
    results = {}
    rows = []

    for store_id in sorted(df["Store"].dropna().unique()):
        sub = df[df["Store"] == store_id]
        out = fit_store_onehot(
            sub,
            steps=steps,
            order=order,
            seasonal=seasonal,
            min_len=min_len,
            agg_sales=agg_sales,
        )
        if out is None:
            continue
        results[int(store_id)] = out
        m = out["metrics"]
        rows.append({"Store": int(store_id), **m})

    metrics_df = pd.DataFrame(rows).set_index("Store").sort_index()
    return results, metrics_df


In [16]:
# Jupyter-safe parallel run with threads
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
from itertools import repeat
import multiprocessing
# print(multiprocessing.cpu_count())  # should return 14

# --- worker (must be top-level in the notebook) ---
def run_store(store_id, steps=54):
    df_store = df[df["Store"] == store_id].copy()
    # df_store = df_store[df_store["Open"] == 1]  # optional
    out = fit_store_onehot(
        df_store,
        steps=steps,
        order=(1, 1, 1),
        seasonal=(1, 1, 1, 7),
        min_len=60
    )
    if out is None:
        return (store_id, None, None)
    return (store_id, out["metrics"], out["forecast"])

# --- driver (call in a cell) ---
start = 0
end = 1115
stores = sorted(df["Store"].dropna().unique())[start:end]   # or all stores
n_threads = multiprocessing.cpu_count()  # tune for your machine

with ThreadPool(n_threads) as pool:
    results = list(pool.starmap(run_store, zip(stores, repeat(42))))

# --- collect ---
lahat_res = []
for store_id, metrics, forecast in results:
    lahat_res.append([store_id, metrics, forecast])

lahat_df = pd.DataFrame(lahat_res)
lahat_df.to_csv("lahat.csv")

/opt/homebrew/Cellar/jupyterlab/4.4.6/libexec/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/Cellar/jupyterlab/4.4.6/libexec/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/Cellar/jupyterlab/4.4.6/libexec/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/homebrew/Cellar/jupyterlab/4.4.6/libexec/lib/python3.13/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho